# **Spotify Top 50 Analizi **

* [Veri Setinin Eklenmesi](#1)
* [Veri Seti İle İlgili İstatistiksel Bilgiler](#2)
* [Şarkıcılar Üzerine Yapılan Görselleştirmeler](#3)
* [Türler Üzerine Yapılan Görselleştirmeler](#4)
* [Nicelikler Üzerine Yapılan Görselleştirmeler
](#5)


 

<a id="1"></a>

## Veri Setinin Eklenmesi

Merhaba ben Kocaeli Üniversitesi Endüstri Mühendisliği 4.Sınıf öğrencisi Enes Polat. Bu çalışmamda Kaggle sitesinde bulunan "Top 50 Spotify Songs -2019" veri setini görselleştirerek açıklamaya çalışacağım. Öncelikle aşağıdaki iki kod ile görselleştirme araçlarımızı aktifleştireceğiz ve veri setimizi çalışma sayfamıza ekleyeceğiz.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from plotly.offline import init_notebook_mode, iplot, plot
import seaborn as sns
import plotly as py
init_notebook_mode(connected=True)
import plotly.graph_objs as go
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
data = pd.read_csv('/kaggle/input/top50spotify2019/top50.csv',encoding='ISO-8859-1')

İngilizce olan sütun başlıklarını daha rahat anlaşılabilmesi için Türkçe olarak değiştirdim.

In [ ]:
data.rename(
            {'Unnamed: 0' : 'Sıralama',
            'Track.Name' : 'MüzikAdı',
            'Artist.Name' : 'ŞarkıcıAdı',
            'Genre' : 'Tür',
            'Beats.Per.Minute' : 'TempoSüresi',
            'Energy' : 'Enerji',
            'Danceability' : 'DansSeviyesi',
             'Loudness..dB..': 'ŞarkıDeğeri',
            'Liveness' : 'Canlılık',
            'Valence.' : 'Olumluluk',
            'Length.': 'Süre',
            'Acousticness..' : 'Akustik',
            'Speechiness.' : 'SözMiktarı',
            'Popularity' : 'Popülerlik'},axis=1,inplace=True)

<a id="2"></a>

## Veri Seti İle İlgili İstatistiksel Bilgiler

Aşağıdaki kısımdaki sütun adlarına baktığımızda Spotify'da dinlenen ilk 5 şarkının adını şarkıcılarını türlerini ve farklı özelliklerinin puanlamalarını görebiliriz.

In [ ]:
data.head()

Bu kod ile kaç sütun ve kaç satır olduğunu görebiliriz. Veri setimizde 50 adet satır ve 14 adet sütun bulunmaktadır. Her bir sütunun adı ise aşağıdaki kod ile gösterilebilmektedir.

In [ ]:
data.shape

In [ ]:
data.columns

Öncelikle veri setimizdeki her sütun için boş veri olup olmadığını incelememiz gerekmektedir. info komutu bunu göstermektedir. 50 adet satırımız vardı ve boş veri olmadığını görmekteyiz.

In [ ]:
data.info()

Bu kod ile veri setindeki her bir sütunun istatistikler değerlerini görmekteyiz.

In [ ]:
print(data.describe())

<a id="3"></a>

## Şarkıcılar İle İlgili Görselleştirmeler

Bu grafik şarkıcıların çıkardıkları şarkıların popülerliklerine göre sıralamasını göstermektedir. Grafiğe göre listedeki en popüler şarkıcı *Bad Bunny*'dir.

In [ ]:
data.head()

In [ ]:
Şarkıcılar = data.ŞarkıcıAdı.unique()
data.Popülerlik = data.Popülerlik.astype(float)
popülerlik_oran_toplam=[]

for i in Şarkıcılar:
    x=data[data['ŞarkıcıAdı']==i]
    popülerlik_oran_bireysel = sum(x.Popülerlik)/len(x)
    popülerlik_oran_toplam.append(popülerlik_oran_bireysel)


popülerlik_data = pd.DataFrame({'Şarkıcı Adı' : Şarkıcılar, 'Popülerlik' : popülerlik_oran_toplam})
popülerlik_index=popülerlik_data['Popülerlik'].sort_values(ascending =False).index.values
popülerlik_sorted_data = popülerlik_data.reindex(popülerlik_index).head(15)

plt.Figure(figsize=(80,45))
sns.barplot(y=popülerlik_sorted_data['Popülerlik'] ,x=popülerlik_sorted_data['Şarkıcı Adı'])
plt.xticks(rotation=90)
plt.show()


Aşağıdaki grafikte listedeki 50 şarkı içerisinde en fazla şarkısı olan şarkıcıların adları daha büyük yazılıdır. Bu grafiğe göre listede en fazla Ed Sheeran'ın şarkısı bulunmaktadır.

In [ ]:
x= data.ŞarkıcıAdı
plt.subplots(figsize=(8,8))
wordcloud = WordCloud(
                          background_color='white',
                          width=512,
                          height=384
                         ).generate(" ".join(x))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('graph.png')

plt.show()

<a id="4"></a>

## Türler Üzerine Yapılan Görselleştirmeler

Şarkıları türlerine göre kıyaslayan aşağıdaki grafiğe göre ise %16'lık bir dilim ile listede en fazla "Dance Pop" türünde şarkı bulunmaktadır. Bunu %14'lük dilim ile "Pop" türü %10'luk dilim ile ise "Latin" türü izlemektedir.

In [ ]:
tür_sayı = data.Tür.value_counts()
türler = data.Tür.unique()
tür_data = [türler,tür_sayı]


labels = data.Tür.value_counts().index
colors = ['grey','blue','red','yellow','green','brown']
explode = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
sizes =  data.Tür.value_counts().values

# visual
plt.figure(figsize = (10,10))
plt.pie(sizes, explode=explode, labels=labels, colors=colors, autopct='%1.1f%%')
plt.title('Tür Dağılımları',color = 'blue',fontsize = 15)
plt.show()

Aşağıdaki kodda bulunan "groupby" kısmı istenilen verilerin gruplanmasını sağlar. Tür verilerinin bilgilerini aşağıdaki tablo bize vermektedir. Her türün veri setindeki nicelikler açısından ortalama puanları verilmiştir.

* Bu tabloya göre en enerjik şarkılar 80 puan ortalaması "reggaeton flow" türündeki şarkılardır.
* İçerisinde en fazla söz geçen şarkılar ise 34 puan ortalaması ile "electropop" türündeki şarkılardır.
* En popüler tür ise 92.5 puan ile "dwf rap" türüdür. Bunu 92 puan ile "electropop " ve "trap music" türleri takip etmektedir.

In [ ]:
tür_ort = data.groupby('Tür').mean()
tür_ort

Burada her bir nicelik için türlerin kendi aralarındaki kıyaslamaya bakabiliriz.

In [ ]:
def plot_tür_analiz(feat):    
    plt.figure(figsize=(8,6))
    sns.barplot(data=data,y=tür_ort.sort_values(by=feat,ascending=False).index,x=tür_ort.sort_values(by=feat,ascending=False)[feat])


In [ ]:
for i in data.select_dtypes(exclude='O').columns:
    plot_tür_analiz(i)

<a id="5"></a>
## Nicelikler Üzerine Yapılan Görselleştirmeler

Önceden de hesapladığımız türlerin nicelik ortalamalarının grafiklere dökülmüş halleri aşağıda gösterilmektedir. Hangi türün her nicelik için kaç puan aldığı buradan da görülmektedir. 

In [ ]:
Tür_Data =[]
Türler = data.Tür.unique()
tempo=[]
dans=[]
canlılık=[]
popülerlik=[]

for i in Türler:
    x= data[data['Tür'] == i]
    Tür_data_oran_tempo = (sum(x.TempoSüresi)/len(x))
    Tür_data_oran_dans = (sum(x.DansSeviyesi)/len(x))
    Tür_data_oran_canlılık = (sum(x.Canlılık)/len(x))
    Tür_data_oran_popülerlik = (sum(x.Popülerlik)/len(x))
    tempo.append(Tür_data_oran_tempo)
    dans.append(Tür_data_oran_dans)
    canlılık.append(Tür_data_oran_canlılık)
    popülerlik.append(Tür_data_oran_popülerlik)

Tür_data=pd.DataFrame({'Türler' : Türler , 'Tempo Oranları' : tempo ,'Dans Edebilme Oranları' : dans ,'Canlılık Oranları' : canlılık,
                       'Popülerlik Oranları' : popülerlik})

trace1 = go.Scatter(
                    x=Tür_data['Türler'],
                    y=Tür_data['Tempo Oranları'],
                    name = 'Tür - Tempo İlişkisi',
                    )
trace2 = go.Scatter(
                    x=Tür_data['Türler'],
                    y=Tür_data['Dans Edebilme Oranları'],
                    name = 'Tür - Dans Edebilme İlişkisi',
                    )
trace3 = go.Scatter(
                    x=Tür_data['Türler'],
                    y=Tür_data['Canlılık Oranları'],
                    name = 'Tür - Canlılık İlişkisi',
                    )
trace4 = go.Scatter(
                    x=Tür_data['Türler'],
                    y=Tür_data['Popülerlik Oranları'],
                    name = 'Tür - Popülerlik İlişkisi',
                    )
İlişki_data = [trace1,trace2,trace3,trace4]


layout = go. Layout(
    dict(title = 'Şarkı Türleri İle Bazı Özelliklerin Değişimi',
              xaxis= dict(title= 'Türler',ticklen= 5,zeroline= False)))


fig =go.Figure(data=İlişki_data , layout=layout)
iplot(fig)


Aşağıdaki grafikte ise her türün Dans ve Canlılık oranlarının karşılaştırılması görülmektedir.

In [ ]:
dans_oranları=[]
canlılık_oranları=[]
for i in türler:
    x=data[data['Tür']==i]
    dans_oranları_bir = (sum(x.DansSeviyesi)/len(x))/max(x.DansSeviyesi)
    canlılık_oranları_bir = (sum(x.Canlılık)/len(x))/max(x.Canlılık)
    canlılık_oranları.append(canlılık_oranları_bir)
    dans_oranları.append(dans_oranları_bir)
enerji_canlılık_data1= pd.DataFrame({'türler' : türler,'dans_oranları' : dans_oranları , 'canlılık_oranları' : canlılık_oranları})

trace1 = go.Scatter(
                    x=enerji_canlılık_data1.türler,
                    y=enerji_canlılık_data1.dans_oranları,
                    mode= 'lines',
                    name = 'Dans Oranları',
                    marker = dict(color = 'rgba(16, 112, 2, 0.8)'),
                    text=enerji_canlılık_data1.dans_oranları)

trace2 = go.Scatter(
                    x=enerji_canlılık_data1.türler,
                    y=enerji_canlılık_data1.canlılık_oranları,
                    mode= 'lines+markers',
                    name = 'Canlılık Oranları',
                    marker = dict(color = 'rgba(250,12, 2, 0.6)'),
                    text=enerji_canlılık_data1.canlılık_oranları)

data1 =[trace1 , trace2]

layout = dict(title = 'Türlere Göre Dans Ve Canlılık Oranları',
              xaxis= dict(title= 'Türler',ticklen= 5,zeroline= False))


fig=dict(data=data1 , layout=layout)
iplot(fig)



Aşağıdaki grafik şarkıların incelenen niceliklerinin birbirleriyle olan korelasyon ilişkisini göstermektedir.Korelasyon ilişkisi iki nicelik arasındaki doğrusal ilişki seviyesine göre belirlenmektedir. Korelasyon katsayısı olan "r" değeri -1 ile +1 arasında değer almaktadır. 
* Katsayının değeri -1 'e yaklaştıkça iki nicelik arasında negatif yönlü bir doğrusal ilişki var demektir. Yani bu iki nicelik birbirini negatif bir şekilde etkiliyordur. 
* +1'e yaklaştıkça ise pozitif yönde bir ilişki oluşmaktadır. 
* Katsayı eğer 0 ise bir ilişki yoktur.

Bu bilgilere göre tablo incelendiği zaman en çok göze çarpan ilişkilerden biri "Tempo Süresi" ile "Söz Miktarı" arasındaki ilişkidir. 0.557 olan r değeri bize şarkının içerisindeki söz miktarının tempo süresini pozitif yönde etkilediğini göstermektedir. 

Aynı şekilde "Olumluluk" ile "Enerji" arasındaki 0.439'luk değer ise bize şarkı ne kadar enerjik ise o kadar olumlu bir etki bırakmakta olduğunu göstermektedir. 

In [ ]:
f,ax=plt.subplots(figsize=(15,15))
sns.heatmap(data.corr(),annot=True,linewidths = 0.5,linecolor = 'white' ,fmt= '.3f',ax=ax )
plt.show()

Aşağıdaki jointplot'da ise yine korelasyon seviyesi görülmektedir. Ancak bu grafikte 2 değer arasındaki korelasyon hesaplanmıştır. Buna göre grafiğe bakıldığında Olumluluk ve Enerji nicelikleri arasında pozitif doğrusal ilişki bulunmaktadır. Yukarıdaki grafikten kontrol ettiğimizde de 0.439 r katsayısı görülmektedir.

In [ ]:
plt.Figure(figsize=(15,15))
sns.jointplot(data.Olumluluk,data.Enerji,color='cyan', kind='kde')
plt.show()

Aşağıdaki grafik listede bulunan şarkıların enerjiklik puanına göre sıralanmasıdır. Her bir dairenin üstüne gelindiğinde şarkının listedeki sırası ve enerjiklik puanı gözükmektedir. 

In [ ]:
Enerji_seviye  = data.Enerji
enerji_color = [float(each) for each in data.Enerji]
data1 = [
    {
        'y': data.Enerji,
        'x': data.Sıralama,
        'mode': 'markers',
        'marker': {
            'color': enerji_color,
            'size': data.Sıralama,
            'showscale': True
        },
        "text" :  data.ŞarkıcıAdı
        
    }
]
iplot(data1)